In [32]:
from azureml.core import Workspace, Experiment, Dataset

ws = Workspace.get(name="quick-starts-ws-120633")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-120633
Azure region: southcentralus
Subscription id: 8e713106-916f-4177-890e-435b90d7adc4
Resource group: aml-quickstarts-120633


In [33]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "AMLComp"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target {}.'.format(cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                               max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

print("Azure Machine Learning Compute attached")

Found existing compute target AMLComp.
Azure Machine Learning Compute attached


In [28]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.95, 1.05),
        "--max_iter": uniform(95, 105)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

import shutil
project_folder = './training'
shutil.copy('train.py', project_folder)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = project_folder,
              entry_script = 'train.py',
              compute_target = compute_target,
              #use_docker=False 
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='accuracy', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=30,
                             max_concurrent_runs=4
)

In [29]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_1dda30c2-73cc-4c22-a451-6c7d1bdc0a1e',
 'target': 'AMLComp',
 'status': 'Canceled',
 'startTimeUtc': '2020-10-14T10:17:34.228454Z',
 'endTimeUtc': '2020-10-14T10:19:36.305392Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'details': [],
   'debugInfo': {'message': '[(https://southcentralus.experiments.azureml.net/subscriptions/8e713106-916f-4177-890e-435b90d7adc4/resourceGroups/aml-quickstarts-120633/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-120633/experiments/**SCRUBBED**/runs/HD_1dda30c2-73cc-4c22-a451-6c7d1bdc0a1e_0, {\'error\': {\n    "additional_properties": {\n        "componentName": null\n    },\n    "error": {\n        "additional_properties": {\n            "messageFormat": null,\n            "messageParameters": {},\n            "referenceCode": null,\n            "severity": null\n        },\n        "code": "UserError",\n        "message": "User program fa

In [30]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n Regularization Strength:',parameter_values[3])
print('\n Max iterations:',parameter_values[5])
print(best_run.get_file_names())

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [31]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = Dataset.Tabular.from_delimited_files(path = web_path)

NameError: name 'Dataset' is not defined

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
df = ds.to_pandas_dataframe(df)
df.head()

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'accuracy',
    training_data = x_train,
    label_column_name = "'y',
    n_cross_validations = 5)

In [ ]:
# Submit your automl run

from azureml.core.experiment import Experiment

experiment = Experiment(ws, "automl_experiment")
automl_run = exp.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)